In [0]:
dbutils.fs.ls('/')

In [0]:
dbutils.fs.ls('/user/hive/warehouse/stores/_delta_log/')

In [0]:
dbutils.fs.head('dbfs:/user/hive/warehouse/stores/_delta_log/00000000000000000001.json')

In [0]:
%sql
create database if not exists databricks_example;


In [0]:
%sql
show databases

In [0]:
%sql
select count(*) from databricks_example.stores



In [0]:
from delta.tables import DeltaTable
delta_store = DeltaTable.forName(spark, "stores")
delta_store.delete()
display(delta_store.history())


In [0]:
countries_df = spark.read.format("delta").option("header", "true").load("dbfs:/user/hive/warehouse/databricks_example.db/countries")
display(countries_df)

In [0]:
countries_1 = countries_df.where("region_id in (10,20,30)")
display(countries_1)
countries_2 = countries_df.where("region_id > 20")
display(countries_2)


In [0]:
countries_1.write.saveAsTable("countries_1")


In [0]:
countries_2.write.saveAsTable("countries_2")

In [0]:
%sql
update countries_2 set name = upper(name)

In [0]:
%sql
 
MERGE INTO countries_1 tgt
USING countries_2 src
ON tgt.country_id = src.country_id
WHEN MATCHED THEN
  UPDATE SET
    tgt.name = src.name
WHEN NOT MATCHED
  THEN INSERT (
    tgt.country_id,
    tgt.name,
    tgt.nationality,
    tgt.country_code,
    tgt.iso_alpha2,
    tgt.capital,
    tgt.population,
    tgt.area_km2,
    tgt.region_id,
    tgt.sub_region_id,
    tgt.intermediate_region_id,
    tgt.organization_region_id
  )
  VALUES (
    src.country_id,
    src.name,
    upper(src.nationality),
    src.country_code,
    src.iso_alpha2,
    src.capital,
    src.population,
    src.area_km2,
    src.region_id,
    src.sub_region_id,
    src.intermediate_region_id,
    src.organization_region_id
  )

In [0]:
display(countries_1)

In [0]:
%sql
select * from countries_1

In [0]:
%sql
describe history countries_1

In [0]:
%sql
select * from countries_1 version as of 0

In [0]:
def divide_cols(a,b):
    return a/b

In [0]:
%sql

select * , divide_cols(population,area_km2) from countries_1

In [0]:
spark.udf.register("divide_cols", divide_cols)